<a href="https://colab.research.google.com/github/kingsgameBen/Lab1VacSkills/blob/main/Lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab1 爬蟲專題 
###人力銀行python職缺-工作技能統計

1. 分別下載1111、104人力銀行python職缺
    儲存成.json、.csv 、.xlsx檔
2. 分析統計英文的技能terminology
3. 依照terms出現次數產生文字雲

# 1111人力銀行爬蟲

In [ ]:
# 1111人力銀行
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import os
# MAC + Pycharm: SSL Certification failed 驗證機制較嚴苛 須證書代碼
import ssl
ssl._create_default_https_context = ssl._create_unverified_context


table = {
        "job_name": [],
        "job_href": [],
        "update_date": [],
        "company": [],
        "com_area": [],
        "industry": [],
        "exp_require": [],
        "edu_require": [],
        "skills": [],
        "dept_require": [],
        "salary": [],
        "job_content": [],
        "job_type": [],
        "benefit_tags": [],
        "benefit_desc": []
    }
# .csv資料夾名稱
DIR = "./1111"


def vacancy_1111():
    BASE_DIR = "/content/drive/MyDrive/Lab1/1111"
    # 計數職缺分頁數、職缺項目數
    pagination = 1
    count = 1

    # 搜尋結果第一頁(20筆),放在while loop 外以避免重複比對執行
    url = "https://www.1111.com.tw/search/job?ks=python"
    response = requests.get(url).text
    html = BeautifulSoup(response, features="html.parser")
    # 取得最後一頁, 3/12網站更改class naming
    # final_page = html.select('div.adv-footer > div.container > div.adv-footer__content > div.adv-footer__content-item > select > option:nth-last-of-type(1)')[0]['value']
    final_page = html.find("div", class_="srh-footer").select(
        'div.container > div.srh-footer__content > div.srh-footer__content-item > select > option')[-1]['value']
    final_page = int(final_page)
    while True:
        if pagination != 1:
            url = "https://www.1111.com.tw/search/job?ks=python&fs=1&page=" + str(pagination) + "&act=load_page"
            response = requests.get(url).text
            response = json.loads(response)['html']
            html = BeautifulSoup(response, features="html.parser")
        print(pagination, '/', final_page)

        rs = html.find_all("div", class_="item__job-info")
        for r in rs:
            # 職缺連結:-> str
            job_href = r.find("a")['href'].strip()
            response = requests.get(job_href).text
            html = BeautifulSoup(response, features="html.parser")
            # 職缺名稱:-> str
            job_name = html.find('div', class_="vacancy-header-row").find('h1').text.strip()
            # 更新日: -> str
            update_date = html.find('div', class_="vacancy-header-row").find('p', class_='update').text.split("：")[-1]
            # 公司名稱:-> str
            company = html.find('div', class_="vacancy-company-text").find('h2').find('a').text.strip()
            # 公司產業別: -> str
            if html.find('div', class_="vacancy-company-text").find('div', class_="company-category"):
                industry = html.find('div', class_="vacancy-company-text").find('div', class_="company-category").find('a').text.strip()
            else:
                # 若顯示頁面找不到產業類別,則從原始碼的特定script標籤中取出
                desc = json.loads(html.find_all("script", {"type": "application/ld+json"})[1].contents[0])[0]
                industry = desc['industry']
            main_contents = html.find('div', class_="vacancy-header-row").find('ul', class_="vacancy-description-main").find_all('p')

            # 取出職缺主要項目( 薪資、 工作經驗、 學歷、 公司地區 ):-> str
            salary, exp_require, edu_require, com_area = "", "", "", ""
            # 用包含式逐項取出職缺主要項目內容依序放入清單的變數中
            [salary, exp_require, edu_require, com_area] = [spans.select('span:nth-of-type(2)')[0].text.strip() for spans in main_contents]

            # 先找到職缺詳細內容的大區塊作為基準點,在依此尋找各項資料
            all_info = html.find('section', id="Job-Detail").find('div', class_="container").find('div', class_="row").find('div', id="job-detail-info")
            # 工作內容
            detail_block = all_info.find('div', class_="job-detail-info").find('div', class_="job-detail-info-content").find_all("p")
            # 用包含式將工作內容區塊的所有 段落標籤 移除: -> list
            job_content = [con.text.strip() for con in detail_block]
            # 工作制度/性質
            type_block = all_info.select(".job-detail-panel > .job-detail-panel-content")[0]
            # 用包裹式取出 職務類別 的清單:-> list
            job_type = [a['title'] for a in type_block.select("dl > dd > span.category > a")]
            # 要求條件
            # 條件標題:-> list
            require_titles = all_info.select(".job-detail-panel > .job-detail-panel-content")[2].find('dl').find_all('dt')
            # print([t.text.strip(':') for t in require_titles])
            require_block = all_info.select(".job-detail-panel > .job-detail-panel-content")[2].find('dl')
            # 科系限制:->str(不拘) or list
            departments = require_block.select('dd > span')[3]
            # 若有科系要求則以 list 存入,不拘 則直接以 str 儲存
            if departments.find_all('a'):
                dept_require = [dept['title'] for dept in departments.find_all('a')]
            else:
                dept_require = departments.text.strip()
            # 其他要求條件(0~多項條件):-> dict
            skills = {}
            if len(require_titles) > 4:
                # 去除 latin1的不間斷空格、\r\n換行、\t縮排、縮減大量空格
                extra_requires = [dd.text.replace('\xa0', '').replace('\r', '').replace('\n', '').replace('\t', '').replace('     ', '').strip() for dd in require_block.select('dt + dd')[4:]]
                for i in require_titles[4:]:
                    idx = i.text.strip('：')
                    skills[idx] = extra_requires[require_titles.index(i)-4]
            # 福利制度
            benefit_tags, benefit_desc = "", ""
            if all_info.find('div', id='service-button-point'):
                benefit_block = all_info.find('div', id='service-button-point').find('div', class_='job-detail-panel-content')
                # 福利制度的標籤(不抓取法定項目的福利):-> list
                if benefit_block.select('a'):
                    benefit_tags = [item['title'] for item in benefit_block.select('a') if item['title'] != 'legal']
                # 其他福利制度的說明:->str
                if benefit_block.select('div.list-box-full > p'):
                    benefit_desc = benefit_block.select('div.list-box-full > p')[0].text.strip()

            table['job_name'].append(job_name)
            table['job_href'].append(job_href)
            table['update_date'].append(update_date)
            table['company'].append(company)
            table['industry'].append(industry)
            table['com_area'].append(com_area)
            table['edu_require'].append(edu_require)
            table['exp_require'].append(exp_require)
            table['salary'].append(salary)
            table['job_content'].append(job_content)
            table['job_type'].append(job_type)
            table['dept_require'].append(dept_require)
            table['skills'].append(skills)
            table['benefit_tags'].append(benefit_tags)
            table['benefit_desc'].append(benefit_desc)

            # print()
            print(count, '職缺：', job_name, job_href)
            # print('工作：')
            # print('更新日', update_date)
            # print('公司,產業別：', company, industry)
            # print('地點：', com_area)
            # print(edu_require)
            # print(exp_require)
            # print(salary)
            # print(job_content)
            # print(job_type)
            # print(dept_require)
            # print(skills)
            # print(benefit_tags)
            # print(benefit_desc)
            print('--'*100)
            count = count + 1

        pagination = pagination + 1
        # 判斷是否超過最終頁, 若超過則跳出for loop 及 while loop
        if pagination > final_page:
            file_name = "vacancy_python"
            if not os.path.exists(BASE_DIR):
                os.makedirs(BASE_DIR)
            df = pd.DataFrame(table)
            csv_fn = BASE_DIR+"/"+file_name+".csv"
            df.to_csv(csv_fn, encoding="utf-8", index=False)
            # 儲存JSON資料(一個JSON檔儲存全部職缺)
            json_fn = BASE_DIR + "/" + file_name + ".json"
            with open(json_fn, "w", encoding="utf-8") as f:
                json.dump(table, f, ensure_ascii=False, indent=4)
            print("="*30, "Procedure End", "="*30)

            excel_fn = BASE_DIR+"/"+file_name+".xlsx"
            df.to_excel(excel_fn)
            count = count - 1
            return count



# 104人力銀行爬蟲

In [ ]:
# 104銀行 python相關職缺
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import os
import time


table = {
        "job_name": [],
        "job_href": [],
        "update_date": [],
        "company": [],
        "com_area": [],
        "industry": [],
        "exp_require": [],
        "edu_require": [],
        "skills": [],
        "dept_require": [],
        "salary": [],
        "job_content": [],
        "job_type": [],
        "benefit_tags": [],
        "benefit_desc": []
    }
DIR = "./104"


def vacancy_104():
    BASE_DIR = "/content/drive/MyDrive/Lab1/104"
    # 計數職缺分頁數、職缺項目數
    pagination = 115
    count = 1
    while True:
        # 只取得工作經歷<1年的python職缺
        url_1year = "https://www.104.com.tw/jobs/search/?keyword=python&page=" + str(pagination) + "&jobexp=1"
        response = requests.get(url_1year).text
        html = BeautifulSoup(response, features="html.parser")

        # 取出第2個沒有src屬性的script標籤,其內容中有totalPage的總頁數資料,
        # 目前無法透過.text或.contents取得totalPage, 改成用while True + 該page取不到ResultSet來終止迴圈
        rs = html.find("div", id="main-content").find("div", id="js-job-content").select("article.job-list-item")
        # 以還能不能取得職缺項目, 判斷是否停止搜尋, 停止while迴圈前儲存所有資料
        if not rs:
            file_name = "vacancy_python"
            csv_fn = BASE_DIR +"/"+file_name+".csv"
            if not os.path.exists(BASE_DIR):
                os.makedirs(BASE_DIR)
            df = pd.DataFrame(table)
            df.to_csv(csv_fn, encoding="utf-8", index=False)
            # 儲存JSON資料(一個JSON檔儲存全部職缺)
            json_fn = BASE_DIR+"/"+file_name+".json"
            with open(json_fn, "w", encoding="utf-8") as f:
                json.dump(table, f, ensure_ascii=False, indent=4)
            print("="*30, "Procedure End", "="*30)

            excel_fn = BASE_DIR+"/"+file_name+".xlsx"
            df.to_excel(excel_fn)
            count = count - 1
            return count

        # start_time = time.time()
        # 建立網址清單,使用grequests的imap()平行發送多個請求
        # href_ls = ["https:" + r.find("a")['href'].strip() for r in rs]
        # reqs = [grequests.get(href) for href in href_ls]
        # responses = grequests.imap(reqs, grequests.Pool(len(reqs)))
        for r in rs:
            # 直接取得父元素下第一個取得的<a>, 職缺連結:-> str
            job_href = "https:" + r.find("a")['href'].strip()
            response = requests.get(job_href).text
            html = BeautifulSoup(response, features="html.parser")
            # print(html)
            titles = html.find("title").text.split("｜")
            # 職缺名稱
            job_name = titles[0].strip()
            # 公司名稱
            company = titles[1].strip()
            # 公司地區
            com_area = titles[2].split("－")[0]
            # 網頁原始碼只有部分資料, 完整資料是透過json帶進來的,做法有別於1111, 須以JSON方式操作
            # 以？分割並透過取段操作取得顯示網址列的職缺ID - 長度4~5碼英數字, 放入職缺內容的JSON真實網址
            json_id = job_href.split("?")[0].split("/")[-1]
            json_url = "https://www.104.com.tw/job/ajax/content/" + json_id
            headers = {
                'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.192 Safari/537.36',
                'Referer': 'https://www.104.com.tw/job/' + json_id + '?jobsource=jolist_a_relevance'
            }

            response = requests.get(json_url, headers=headers)
            json_data = json.loads(response.text)["data"]
            # 更新日期、產業別、學歷條件、工作經驗、薪資、工作內容、職務類別、科系條件、工作技能、福利制度
            update_date = json_data["header"]["appearDate"]
            industry = json_data["industry"]
            edu_require = json_data["condition"]["edu"]
            exp_require = json_data["condition"]["workExp"]
            salary = json_data["jobDetail"]["salary"]
            job_content = json_data["jobDetail"]["jobDescription"].strip()
            job_type = [cate["description"] for cate in json_data["jobDetail"]["jobCategory"]]
            dept_require = json_data["condition"]["major"]
            skills = [special["description"] for special in json_data["condition"]["specialty"]]
            benefit_desc = json_data["welfare"]["welfare"]

            table['job_name'].append(job_name)
            table['job_href'].append(job_href)
            table['update_date'].append(update_date)
            table['company'].append(company)
            table['industry'].append(industry)
            table['com_area'].append(com_area)
            table['edu_require'].append(edu_require)
            table['exp_require'].append(exp_require)
            table['salary'].append(salary)
            table['job_content'].append(job_content)
            table['job_type'].append(job_type)
            table['dept_require'].append(dept_require)
            table['skills'].append(skills)
            table['benefit_tags'].append(None)
            table['benefit_desc'].append(benefit_desc)

            # print()
            print('Page:', pagination)
            print(count, job_name)
            # print(job_href)
            # print(update_date)
            # print('公司,產業別：', company, industry)
            # print('地點：', com_area)
            # print(edu_require)
            # print(exp_require)
            # print(salary)
            # print(job_content)
            # print(job_type)
            # print(dept_require)
            # print(skills)
            # print(benefit_desc)
            print('-' * 100)
            count = count + 1

        pagination = pagination + 1



# main.py主程式
分別呼叫1111、104爬蟲程式

In [ ]:
# 抓取人力銀行職缺, 分析個別工作技能清單
# import bank_104
# import bank_1111
import time
from urllib import request
import pandas as pd
import re
import jieba
from jieba import analyse
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import imageio
import matplotlib.pyplot as plt
from collections import Counter
import os

# The File size(>4.25MB) exceeds configured limit (2.56MB), code insight features not available
# 日誌文件過大,超過默認設定,要增加限制容量並重啟pycharm ->Help->Edit Custom Properties->加入"idea.max.intellisense.filesize=99999"

BASE_DIR = "/content/drive/MyDrive/2021_Python班/專題_爬蟲/Lab1"

# 爬取人力銀行(1111、104)職缺資料,儲存成csv、excel及json資料
# 抓取104人力銀行職缺(工作經歷1年以下):
def find_vacancy(bank):
    start_time = time.time()
    if bank == "104":
        # vac_qty = bank_104.vacancy_104()
        vac_qty = vacancy_104()
    elif bank == "1111":
        # vac_qty = bank_1111.vacancy_1111()
        vac_qty = vacancy_1111()
    end_time = time.time()
    print("Download ", vac_qty, "vacancies of", bank, ". It takes", round((end_time - start_time)/60, 2), 'minutes.')


# 計算詞頻
def count_seg_freq(seg_list):
    seg_df = pd.DataFrame(seg_list, columns=['Terminology'])
    seg_df['count'] = 1
    sef_freq = seg_df.groupby('Terminology')['count'].sum().sort_values(ascending=False)
    sef_freq = pd.DataFrame(sef_freq)
    return sef_freq


# 分析關鍵字
def analyse_kw(mode=0, *mode_item):
    # mode_dic = {0: '菜鳥', 1: '產業別', 2: '學士以下', 3: '不限科系', 4: '薪水4萬以下或面議', 5: '縣市別'}
    url = "https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big"
    request.urlretrieve(url, "dict.big")
    jieba.set_dictionary("dict.big")
    jieba.load_userdict(BASE_DIR + "/Dict/mix_terminologies.txt")

    total_vac = 0
    eng_kw = []
    bank_ls = ["1111", "104"]
    # 不同的人力銀行分別分析
    for bank in bank_ls:
        # fn = bank + "/vacancy_python.json"
        fn = "/content/drive/MyDrive/Lab1/" + bank + "/vacancy_python.json"
        kws = set()
        skill_all = []
        count = 1
        with open(fn, "r", encoding="utf-8") as f:
            # 完整顯示所有資料列
            pd.set_option('display.max_rows', None)
            vacancy = pd.read_json(f)
            df = pd.DataFrame(vacancy)

        if mode == 1:
            if mode_item:
                mode_item = str(mode_item[0])
                mask1 = df["industry"].str.contains(mode_item)
                mask2 = df["industry"].notnull()
                df = df[(mask1 & mask2)]
            else:
                mask1 = df["industry"].notnull()
                df = df[mask1]
        elif mode == 2:
            mask1 = df["edu_require"].str.contains("不拘|高中|專科|大學")
            mask2 = df["edu_require"].isnull()
            df = df[(mask1 | mask2)]
        elif mode == 3:
            mask1 = df["dept_require"].str.contains("不拘|商業|其他|一般|普通|軟體|網路")
            mask2 = df["dept_require"].isna()
            if bank == "104":
                mask2 = df["dept_require"].str.len() == 0
                df = df[mask2]
            else:
                df = df[(mask1 | mask2)]
        elif mode == 4:
            mask1 = df["salary"].str.contains("^月薪[23].,.*")
            mask2 = df["salary"].str.contains("面議")
            # mask2 = df["salary"].str.contains("年薪[4]??,*")
            df = df[(mask1 | mask2)]
        elif mode == 5:
            if mode_item:
                mode_item = str(mode_item[0])
                mask1 = df["com_area"].str.contains(mode_item)
            else:
                mask1 = df["com_area"].str.contains("台北市")
            df = df[mask1]
        print(bank+":", df.shape)
        print("_"*100)

        if bank == '1111':
            df.set_index("exp_require", inplace=True)
            if mode == 0:
                # 嘗試篩選出工作經驗2年以下及不拘的職缺
                df = df.loc[["工作經歷-不拘", "1年以上工作經驗"]]

            total_vac += len(df.index)
            for skill in df['skills']:
                if type(skill) == dict:
                    # 目前先過濾掉語言能力的要求(匹配忽略大小寫)
                    values = [val for k, val in skill.items() if k != '語言能力']  # and not re.match(r'[a-z]+', val, re.I)]
                else:
                    values = skill
                if not skill or not values:  # 略過空資料
                    continue
                skill_all += values
        elif bank == '104':
            total_vac += len(df.index)
            for sk in df['skills']:
                if sk:
                    skill_all += sk

        for val in skill_all:
            val = val.strip().strip("..").strip("...").upper()
            val = val.replace("FAMILIAR", "").replace("COMPUTER", "").replace("SYSTEM", "").replace("WEB", "").replace("EXPERIENCE", "")
            val = val.replace("ABILITY", "").replace("JOB", "").replace("KNOWLEDGE", "").replace("SUCH", "").replace("TOOLS", "").replace("SIMILAR", "")
            val = val.replace("MS SQL", "MSSQL")
            jieba.analyse.set_stop_words(BASE_DIR + "/Dict/stopwords.txt")
            jieba.add_word("MACHINE LEARNING")
            jieba.add_word("DEEP LEARNING")
            jieba.add_word("NODE.JS")
            jieba.suggest_freq("NODE.JS", True)
            if bank == '1111':
                val = re.sub(r"[Website]*?\(?[http?]\S+[/)$|/$]", "", val, flags=re.IGNORECASE)  # 刪除連結URL
                keywords = jieba.analyse.extract_tags(val, topK=30)
                # print("\n關鍵詞:", count,  keywords)
                for kw in keywords:
                    if re.match(r'[A-Z]+', kw):
                        eng_kw.append(kw)
                    kws.add(kw)
            elif bank == '104':
                if re.match(r'[A-Z]+', val):
                    eng_kw.append(val)

            count = count + 1

    word_freq = count_seg_freq(eng_kw)
    freq_percent = round(word_freq[:200]/len(eng_kw)*100, 1)
    freq_percent = freq_percent.rename(columns={"count": "%"})
    word_freq = pd.concat([word_freq, freq_percent], sort=False, axis=1)
    word_freq = word_freq.reset_index()
    print(word_freq[:50])
    print("Total terms:", len(word_freq))
    print("Filtered Vacancy:", total_vac)
    return eng_kw


# 產生文字雲
def word_cloud(mode=0, *mode_item):
    eng_kw = analyse_kw(mode, *mode_item)
    font_path = BASE_DIR + '/font/PatuaOne.ttf'
    show_words = 200

    # Default mode's image
    back_path = BASE_DIR + '/Image/github.jpg'
    fn_type = "fresh_0_"
    if mode == 1:
        back_path = BASE_DIR + '/Image/github2.jpeg'
        fn_type = "indus_1_"
        show_words = 150
        if mode_item:
            fn_type += mode_item[0] + "_"
            show_words = 100
    elif mode == 2:
        back_path = BASE_DIR + '/Image/explosion.png'
        fn_type = "edu_2_"
        show_words = 150
    elif mode == 3:
        back_path = BASE_DIR + '/Image/riot.jpg'
        fn_type = "dept_3_"
        show_words = 100
    elif mode == 4:
        back_path = BASE_DIR + '/Image/python.jpg'
        fn_type = "pay_4_"
        show_words = 100
    elif mode == 5:
        back_path = BASE_DIR + '/Image/umu.jpeg'
        fn_type = "area_5_"
        show_words = 150
        if mode_item:
            fn_type += mode_item[0] + "_"
            show_words = 100

    fn = fn_type + back_path.split("/")[-1].split(".")[0]+".png"
    back_color = imageio.imread(back_path)
    if not eng_kw:
        eng_kw.append('EMPTY RESULT.查無職缺')
    wc = WordCloud(
        max_words=show_words,
        mask=back_color,
        font_path=font_path,
        repeat=False,
        collocations=False
    ).generate_from_frequencies(Counter(eng_kw))
    if mode in [2, 3, 4, 5]:
        image_colors = ImageColorGenerator(back_color)
        wc.recolor(color_func=image_colors)

    if mode in [1, 5]:
        plt.rcParams['font.sans-serif'] = 'STHeiti'
    plt.legend(loc=1, title=fn[:-4], labels=fn_type.split("_")[0])
    plt.imshow(wc)
    plt.axis("off")
    plt.show()
    if not os.path.exists("wordcloud_result"):
        os.mkdir("wordcloud_result")
    fig_path = "wordcloud_result/" + fn
    wc.to_file(fig_path)  # 輸出圖片


if __name__ == '__main__':
    # 2310筆 約26分,
    # find_vacancy("104")
    # 893筆 約 15.6分, 908筆 約13分
    # colab下載899筆 約5分鐘 3/20: 905筆約28分
    # find_vacancy("1111")

    # mode_dic = {0: '菜鳥', 1: '產業別', 2: '學士以下', 3: '不限科系', 4: '薪水4萬以下或面議', 5: '縣市別'}
    # print(analyse_kw())
    # word_cloud()
    # word_cloud(1)  # => 全部產業的職缺, terms統計前150項
    # word_cloud(1, "電腦")  # => 產業別有“電腦” 的職缺, terms統計前100項
    # word_cloud(1, "光電|電子")  # => 產業別有“光電”或“電子” 的職缺, terms統計前100項
    # word_cloud(2)
    # word_cloud(3)
    # word_cloud(4)
    # word_cloud(5)
    word_cloud(5, "桃園")


# 工具程式 - 字典清單下載整併

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import os
import pandas as pd

# 從wikipedia取得程式語言清單關鍵字
def programming_lang():
    url = "https://zh.wikipedia.org/wiki/%E7%BC%96%E7%A8%8B%E8%AF%AD%E8%A8%80%E5%88%97%E8%A1%A8"
    response = urlopen(url)
    html = BeautifulSoup(response, features="html.parser")
    main_content = html.find('div', id='bodyContent').find('div', id='mw-content-text').find_all('div', class_='div-col')
    a_ls = [tag.find_all('a') for tag in main_content]
    programming_list = []
    for anchors in a_ls:
        for lang in anchors:
            programming_list.append(lang.text)
    if not os.path.exists('Dict'):
        os.makedirs('Dict')
    with open('Dict/programming.dict', 'w', encoding='utf-8')as f:
        for text in programming_list:
            f.write(text+'\n')


# 因為字詞來源於中國網站, 簡體字轉換成繁體字的操作, 暫時以手動方式轉換
def operating_sys():
    url = "https://zh.wikipedia.org/wiki/%E6%93%8D%E4%BD%9C%E7%B3%BB%E7%BB%9F%E5%88%97%E8%A1%A8"
    response = urlopen(url)
    html = BeautifulSoup(response, features="html.parser")
    main_content = html.find('div', id='bodyContent').find('div', id='mw-content-text').find_all('ul')
    a_ls = [li.find_all('a') for li in main_content]  # if not li.find('span')]  #不排除系統分類標題名稱
    os_ls = []
    for a in a_ls:
        for system in a:
            os_ls.append(system.text)
    if not os.path.exists('Dict'):
        os.makedirs('Dict')
    with open('Dict/operating_sys.dict', 'w', encoding='utf-8')as f:
        for text in os_ls:
            f.write(text + '\n')


# 把字典內容去掉重複並全部轉成大寫
def convert2upperset(file_name):
    file_name = 'Dict/' + file_name + '.dict'
    temp_ls = []
    with open(file_name, 'r', encoding='utf-8')as f:
        temp_ls = f.readlines()
    upper_ls = set()
    for temp in temp_ls:
        upper_ls.add(temp.upper())
    with open(file_name, 'w', encoding='utf-8')as f:
        f.writelines(upper_ls)


# 產業別列表
def industry_list():
    industry_set = set()
    for bank in ["1111", "104"]:
        file_name = "../" + bank + "/vacancy_python.json"
        with open(file_name, 'r', encoding='utf-8')as f:
            vacancy = pd.read_json(f)
        for indus in vacancy['industry']:
            industry_set.add(indus)
        print(bank, ":\n", industry_set, "\n AccumulatedQty:", len(industry_set))
    if not os.path.exists("Dict"):
        os.makedirs("Dict")
    with open("Dict/industry_list.txt", 'w', encoding='utf-8')as f:
        for ind in industry_set:
            if ind:  # 避免寫入None
                f.write(ind+'\n')
    print(len(industry_set))


# 學歷要求列表
def industry_list():
    edu_set = set()
    for bank in ["1111", "104"]:
        file_name = "../" + bank + "/vacancy_python.json"
        with open(file_name, 'r', encoding='utf-8')as f:
            vacancy = pd.read_json(f)
        for indus in vacancy['edu_require']:
            edu_set.add(indus)
        print(bank, ":\n", edu_set, "\n AccumulatedQty:", len(edu_set))
    if not os.path.exists("Dict"):
        os.makedirs("Dict")
    edu_ls = sorted(edu_set)
    with open("Dict/education_list.txt", 'w', encoding='utf-8')as f:
        for edu in edu_ls:
            if edu:  # 避免寫入None
                f.write(edu+'\n')
    print(len(edu_ls))


# 科系要求列表
def dept_list():
    dept_set = set()
    for bank in ["1111", "104"]:
        file_name = "../" + bank + "/vacancy_python.json"
        with open(file_name, 'r', encoding='utf-8')as f:
            vacancy = pd.read_json(f)
        for dept in vacancy['dept_require']:
            for d in dept:
                dept_set.add(d)
        print(bank, ":\n", dept_set, "\n AccumulatedQty:", len(dept_set))
    if not os.path.exists("Dict"):
        os.makedirs("Dict")
    dept_ls = sorted(dept_set)
    with open("Dict/department_list.txt", 'w', encoding='utf-8')as f:
        for dept in dept_ls:
            f.write(dept+'\n')
    print(len(dept_ls))


# 所有技能列表(from人力銀行的技能條件)
def skills_list():
    skills_set = set()
    for bank in ["1111", "104"]:
        file_name = "../" + bank + "/vacancy_python.json"
        with open(file_name, 'r', encoding='utf-8')as f:
            vacancy = pd.read_json(f)
        for skills in vacancy['skills']:
            for skill in skills:
                skills_set.add(skill.strip().upper())
        print(bank, ":\n", skills_set, "\n AccumulatedQty:", len(skills_set))

    if not os.path.exists("Dict"):
        os.makedirs("Dict")
    skills_ls = sorted(skills_set)
    with open("Dict/terminology_list.txt", 'w', encoding='utf-8')as f:
        for skill in skills_ls:
            if skill:  # 避免寫入None
                f.write(skill+'\n')
    print(len(skills_ls))


# 整合成一份所有技能列表(programming.dict、operating_sys.dict、terminology_list.txt => mix_terminologies.txt)
def integrated_term_list():
    terms_set = set()
    for dic in ["operating_sys.dict", "programming.dict", "terminology_list.txt"]:
        file_name = "Dict/" + dic
        with open(file_name, 'r', encoding='utf-8')as f:
            terms_ls = f.readlines()
        for term in terms_ls:
            terms_set.add(term.strip("\n").upper())
        print(dic, ":\n", terms_set, "\n AccumulatedQty:", len(terms_set))

    terms_ls = sorted(terms_set)
    with open("Dict/mix_terminologies.txt", 'w', encoding='utf-8')as f:
        for term in terms_ls:
            if term:  # 避免寫入None
                f.write(term+'\n')
    print(len(terms_ls))


# 薪資待遇列表
def sal_list():
    sal_set = set()
    for bank in ["1111", "104"]:
        file_name = "../" + bank + "/vacancy_python.json"
        with open(file_name, 'r', encoding='utf-8')as f:
            vacancy = pd.read_json(f)
        for indus in vacancy['salary']:
            sal_set.add(indus)
        print(bank, ":\n", sal_set, "\n AccumulatedQty:", len(sal_set))
    sal_ls = sorted(sal_set)
    with open("Dict/salary_list.txt", 'w', encoding='utf-8')as f:
        for sal in sal_ls:
            if sal:  # 避免寫入None
                f.write(sal+'\n')
    print(len(sal_ls))

